In [2]:
from csv import reader
cards = []

with open('./card_list.csv') as f:
    pamreader = reader(f, delimiter=',', quotechar='"')
    for row in pamreader:
        cards.extend([card for card in row if card.strip() and "(" in card])


In [5]:
import os
import re

to_buy = []
not_found = []
not_parsed = []
# 1 Ertai Resurrected <showcase> [DMU] (F)
reg = r"^(?:(?P<count>\d+) )?(?P<name>[^\<\[\(]*)\b(?:(?: <(?P<chevron>[^\<\[\(]*)>)|(?: \[(?P<bracket>[^\<\[\(]*)\])|(?: \((?P<parenthesis>[^\<\[\(]*)\)))*"
wants = os.listdir('./wants')
for want in wants:
    with open(f'./wants/{want}') as f:
        for line in f:
            if not line.strip():
                continue
            # print(line)
            matches = re.match(reg, line)
            if not matches:
                print(f"Could not parse {line}")
                not_parsed.append(line)
                continue
            
            count, name, special, card_set, other = matches.groups()
            for card_name in cards:
                only_name, edition = card_name.split(' (', 1)
                if name == only_name:
                    to_buy.append((int(count), card_name))
                    break
            else:
                not_found.append((int(count), f"{name} ({card_set})"))



In [6]:
not_parsed

[]

In [72]:
def aggregate_count(cards):
    counts = {}
    for count, card in cards:
        if card in counts:
            counts[card] += count
        else:
            counts[card] = count
    return counts.items()

In [77]:
len(to_buy), len(not_found)

(139, 56)

In [73]:
not_found = aggregate_count(not_found)

In [74]:
to_buy = aggregate_count(to_buy)

In [78]:
from csv import writer

with open('./to_buy.csv', 'w') as f:
    pamwriter = writer(f, delimiter=',', quotechar='"')
    
    pamwriter.writerow(["Existing Cards"])
    pamwriter.writerow(["Count", "Card Name"])
    for name, count in sorted(to_buy, key=lambda x: x[0]):
        pamwriter.writerow([count, name])
    
    pamwriter.writerow([])
    pamwriter.writerow(["Not found in list"])
    pamwriter.writerow(["Count", "Card Name"])
    for name, count in sorted(not_found, key=lambda x: x[0]):
        pamwriter.writerow([count, name])